### 구글 코랩 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 라벨값 추출 코드
- 주의!!! 라벨에 해당하는 폴더(ex : ang) 생성 후 %cd (감정) 해서 들어간 뒤에 실행

In [ ]:
%cd /content/drive/MyDrive/AI_Emotion/Train_data/Training/yolo_datasets/labels/face_val

/content/drive/MyDrive/AI_Emotion/Train_data/Training/yolo_datasets/labels/face_val


- ImageOps.Contain을 사용하기 위해서 Pillow==9.1.0을 설치해야 함

In [ ]:
!pip install Pillow==9.1.0

     |████████████████████████████████| 4.3 MB 8.2 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import os
from PIL import Image, ImageOps, ExifTags
image_path = '/content/drive/MyDrive/AI_Emotion/Train_data/Training/WIDER_datasets/images_old/val/'
for f_name in os.listdir(image_path):
  image1 = Image.open(image_path+f_name)
  image1 = ImageOps.exif_transpose(image1)
  image1 = ImageOps.contain(image1, (640,640))
  image1.save('/content/drive/MyDrive/AI_Emotion/Train_data/Training/WIDER_datasets/images/val/'+f_name,format=image1.format, optimize=True, quality=90)

In [ ]:
import os
import cv2
import pandas as pd
from PIL import Image ,ImageOps , ExifTags
from tqdm import tqdm
import traceback

emotion_name = 'wou'
image_path = f'../../../backup/images/{emotion_name}/'
df = pd.read_json(f'../../../backup/labels/json/{emotion_name}_prof.json')
filenames = df["filename"].tolist()

class_num = 6
labels = []
n = 1

for f_name in os.listdir(image_path):
  #1000개까지만 추출
  if n == 1001:
    break
  
  # json 파일에서 갖고 있는 파일들만 추출
  if df['filename'].str.contains(f_name.split("_")[-1]).any():
    file_name_without_jpg = f'{emotion_name}_{str(n).zfill(5)}'
    f = open(file_name_without_jpg+'.txt', 'w')
    #image load
    image1 = Image.open(image_path+f_name)
    

    imag1_size = image1.size

    w_img, h_img = imag1_size[0], imag1_size[1]

    image1 = ImageOps.exif_transpose(image1)

    image1 = ImageOps.contain(image1, (640,640))

    imag1_size = image1.size
    
    image1.save(f'/content/drive/MyDrive/AI_Emotion/Train_data/Training/yolo_datasets_v2/images/face_train/{emotion_name}_' + str(n).zfill(5) +'.jpg',format=image1.format, optimize=True, quality=90)

    # 라벨 데이터값 추출
    index = df[df['filename'].str.contains(f_name.split("_")[-1])].index[0]
    maxX = df['annot_A'][index]['boxes']['maxX']
    minY = df['annot_A'][index]['boxes']['minY']
    minX = df['annot_A'][index]['boxes']['minX']
    maxY = df['annot_A'][index]['boxes']['maxY']
    w = maxX - minX
    h = maxY - minY
    
    # 이미지 shape 로드
    #img_shape = cv2.imread(image_path+f_name).shape
    #w_img, h_img = img_shape[0], img_shape[1]

    # 라벨값 계산
    xcenter = round((minX + w/2) / w_img,6)
    ycenter = round((minY + h/2) / h_img,6)
    w = round(w / w_img,6)
    h = round(h / h_img,6)
    f.write(' '.join([str(class_num), str(xcenter), str(ycenter), str(w), str(h)]))
    print(f'{emotion_name}_{str(n).zfill(5)}',xcenter, ycenter, w, h)
    f.close()
    n += 1
    
else:
  print(f'{n}개의 이미지 라벨링 작업을 완료하였습니다.')

wou_00001 0.516127 0.47372 0.23689 0.388193
wou_00002 0.655105 0.401651 0.289946 0.563668
wou_00003 0.425316 0.413683 0.269113 0.437272
wou_00004 0.449948 0.480658 0.2711 0.614707
wou_00005 0.513133 0.375862 0.272845 0.524138
wou_00006 0.495997 0.429143 0.263363 0.440034
wou_00007 0.47948 0.484738 0.224966 0.524534
wou_00008 0.50404 0.479576 0.294556 0.525187
wou_00009 0.329937 0.418517 0.296057 0.492442
wou_00010 0.418056 0.526461 0.291333 0.612798
wou_00011 0.523574 0.525421 0.286807 0.4632
wou_00012 0.615512 0.410042 0.25127 0.593846
wou_00013 0.42402 0.456486 0.262079 0.543306
wou_00014 0.542167 0.544 0.228 0.4064
wou_00015 0.542329 0.444839 0.249263 0.448077
wou_00016 0.601574 0.493117 0.297371 0.515996
wou_00017 0.507759 0.655964 0.262244 0.626144
wou_00018 0.529831 0.456565 0.220434 0.375105
wou_00019 0.319785 0.403618 0.277375 0.542088
wou_00020 0.538435 0.433736 0.259648 0.467675
wou_00021 0.498824 0.461565 0.300478 0.539169
wou_00022 0.544111 0.525565 0.248138 0.41666
wou_000

### 라벨 이상치 검출기
- 디렉토리 안에서 xcenter, ycenter, w, h이 0보다 작거나 1보다 큰 값을 가지는 라벨을 검출

In [ ]:
import os
weird_files = []

for f_name in os.listdir('/content/drive/MyDrive/AI_Emotion/Train_data/Training/yolo_datasets_v2/labels/face_val'):
	f = open(f_name, 'r')
	line = f.readline()
	values = line.split(" ")[1:]
	for value in values:
		if float(value) >= 1.0 or float(value) <= 0.0:
			weird_files.append(f_name)
			print(f_name, values)
			break
	f.close()